\begin{align*}
u_{tt} &= u_{xx} \\
u_{tt}(t,x) &= \frac{u(t,x+ \Delta x)-2u(t,x)+u(t,x- \Delta x)}{\Delta x^{2}} \\
u_{tt}(t,x) + \frac{2}{\Delta x^{2}}u(t,x) &= \frac{u(t,x+ \Delta x)+u(t,x- \Delta x)}{\Delta x^{2}} \\
\end{align*}
Introduce: $q = \frac{\sqrt{2}}{\Delta x}$.
\begin{align*}
u_{tt}(t,x) + q^{2}u(t,x) &= q^{2}\frac{u(t,x+ \Delta x)+u(t,x- \Delta x)}{2} \\
\end{align*}
Green's functions:

\begin{align*}
u(t,x) = \cos\left(\sqrt{q} t\right) u(0,t) + \frac{\sin\left(\sqrt{q}t\right)}{\sqrt{q}} u_t(0,x) +    
\frac{\sqrt{q}}{2} \int_{0}^{t} \sin\left(\sqrt{q}(s-t) \right) \left[ u(s,x+\Delta x) +u(s,x-\Delta x) \right] ds 
\end{align*}



In [7]:
from random import random as U
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets

# def u(t,x): return x**2 + 2*t #needs to satisfy wave equation
# def ut(t,x): return  2 #needs to satisfy wave equation

# def u(t,x): return x*t**2 + x**3/3 #needs to satisfy wave equation
# def ut(t,x): return 2*x*t  #needs to satisfy wave equation

def u(t,x): return np.sin((x+t)) #needs to satisfy wave equation
def ut(t,x): return np.cos((x+t))  #needs to satisfy wave equation

def P1(t,q): return np.cos( np.sqrt(q)*t) 
def P2(t,q): return np.sin(np.sqrt(q)*t)/np.sqrt(q) 
def GG(t,s,q): return np.sqrt(q)*np.sin(np.sqrt(q)*(t-s)) # not sure this is good ? 
count = 0

def Z0(t,x,dx,dt):
    q =2/dx**2
    s = t*U()
    xnew = x+dx if U()<1/2 else x-dx
    sol = P1(t,q)*u(0,x)
    sol += P2(t,q)*ut(0,x)
    if t/dt>1:
        sol+=t*GG(t,s,q)*Z0(s,xnew,dx,dt)
    elif U()<t/dt:
        sol+=dt*GG(t,s,q)*Z0(s,xnew,dx,dt)
    return sol

def Z(t,x,dx,dt):
    if t< dt: return Z0(t,x,dx,dt) 
    t0 = t-dt
    q =np.sqrt(2)/dx
    s = dt*U()+t0
    xnew = x+dx if U()<1/2 else x-dx
    samp = 5/10
    if U()<samp: 
        sol = P1(dt,q)*Z(t0,x,dx,dt)/samp
    else:
        sol=dt*GG(t,s,q)*q**2*Z(s,xnew,dx,dt)/(1-samp)
    
    rr_rate = q if 1/q <1 else 1

    if 1/rr_rate>U():
        rr = rr_rate
        global count
        count +=1
        sol += rr*dt*P2(dt,q)*ut(t0,x) 
        sol += rr*dt*GG(t,s,q)* 2 *q *ut(s,x)

    return sol

def update_histogram(t,x,dt, dx,nsim):
    nsim = 10**int(nsim)
    global count
    count = 0
    zz = np.array([Z0(t, x, dx,dt) for _ in range(nsim)])
    avg_label.value = f"Updated Average of zz: {np.average(zz):.2f}, {u(t,x):.2f}"
    counter.value = f"Updated Average of count: {count/nsim} "
    plt.hist(zz, bins=50)
    plt.xlabel("Values of Z")
    plt.ylabel("Frequency")
    plt.title("Histogram of Z")
    plt.show()

dt_slider = widgets.FloatSlider(value=1, min=0.1, max=10, step=0.01, description='dt:')
dx_slider = widgets.FloatSlider(value=1, min=0.1, max=10, step=0.01, description='dx:')
t_slider = widgets.FloatSlider(value=0, min=0, max=10.0, step=0.1, description='t:')
x_slider = widgets.FloatSlider(value=0, min=-50, max=50, step=1, description='x:')
nsim_slider = widgets.FloatSlider(value=0, min=0, max=5, step=1, description='nsim:')

avg_label = widgets.Label(value="")
counter = widgets.Label(value="")

display(avg_label)
display(counter)

interact(update_histogram,t=t_slider,x=x_slider, dt=dt_slider, dx=dx_slider, nsim = nsim_slider)

Label(value='')

Label(value='')

interactive(children=(FloatSlider(value=0.0, description='t:', max=10.0), FloatSlider(value=0.0, description='…

<function __main__.update_histogram(t, x, dt, dx, nsim)>

move a period and take the difference

\begin{align*}
u(t+\frac{2 \pi}{\sqrt{q}},x)  = u(t,x) +
\frac{\sqrt{q}}{2} \int_{t}^{t+\frac{2 \pi}{\sqrt{q}} } \sin\left(\sqrt{q}(s-t) \right) \left[ u(s,x+\Delta x) +u(s,x-\Delta x) \right] ds 
\end{align*}
